<a href="https://colab.research.google.com/github/malihasaeed/RAG/blob/main/Copy_of_RAG_assignment1_atomcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain
# Requires Python 3.10+

In [2]:
!pip install -U groq langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.7 MB/s eta 0:00:00


In [3]:
!pip install -qU \
    langchain \
    langchain-community \
    langchain-core \
    langchain-groq \
    faiss-cpu \
    pypdf \
    gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
import os
os.environ["GROQ_API_KEY"] = "rag_chatbot"


In [6]:
#This code saves your secret API key, then creates a smart-robot helper (llm) so you can ask it questions.

from google.colab import userdata
import os

# read your secret by its name
groq_key = userdata.get("rag_chatbot")

# set the official env var Groq expects
os.environ["GROQ_API_KEY"] = groq_key

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile")

# llm.invoke(...) is a function (a built-in action) that sends your question to the AI and brings back the answer, which is then printed
response = llm.invoke("Explain retrieval-augmented generation in simple words.")
#.content pulls out ONLY the words the AI wrote so you can print them.
print(response.content)


Retrieval-augmented generation is a technique used in artificial intelligence (AI) to improve the quality and accuracy of generated text, such as articles, stories, or chatbot responses.

Here's how it works in simple words:

1. **Storage of Knowledge**: A large database or memory is created, which stores a vast amount of text, information, and knowledge on various topics.
2. **Query and Retrieval**: When a user asks a question or requests information, the AI system sends a query to the database to retrieve relevant and useful information related to the request.
3. **Generation**: The retrieved information is then used to generate a response, such as a sentence, paragraph, or article. The AI system uses this information to create new text that is accurate, informative, and relevant to the user's request.
4. **Augmentation**: The generated text is "augmented" with the retrieved information, which means it is enhanced and improved by incorporating the relevant knowledge and facts from th

In [7]:
from google.colab import files
uploaded = files.upload()


Saving business proposal.pdf to business proposal.pdf


In [8]:
#This is standard Python code commonly used in tutorials / documentation to read PDFs using PyPDF2.
#It’s not specific to any AI tool.
#converting the doc into text
!pip install PyPDF2

import PyPDF2

pdf_file = open("business proposal.pdf", "rb")
reader = PyPDF2.PdfReader(pdf_file)

text = ""

for page in reader.pages:
    text += page.extract_text()

pdf_file.close()

print(text[:500])  # show first 500 chars


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.3 MB/s eta 0:00:00
SolviTech — AI & Technology Consulting Proposal  
1. Company Snapshot  
SolviTech  is a technology and AI consulting firm that helps businesses automate operations, 
improve digital presence, and scale customer acquisition  using practical, result -driven AI 
solutions.  
2. Vision & Mission  
Vision  Mission  
Empower businesses with affordable, 
intelligent, scalable  technology 
solutions.  Deliver reliable AI and tech solutions  that automate 
workflows, generate leads, and support sustainab


In [9]:
import langchain
print(langchain.__version__)


1.2.0


In [10]:
import langchain
print(langchain.__version__)


1.2.0


In [11]:
!pip install -q langchain langchain-text-splitters langchain-core


what logic is being used?

🔹 PDFs can be VERY long

🔹 LLMs cannot read huge text at once

🔹 So we break it into small readable pieces (chunks)

This makes RAG / chatbots work better and faster 🚀

In [12]:
#This is a tool from LangChain that breaks long text into smaller pieces (chunks).
from langchain_text_splitters import RecursiveCharacterTextSplitter
#This wraps your text into a Document object — a neat container LangChain likes to work with.text=documemt box
from langchain_core.documents import Document

# Wrap your extracted text into a Document
#Creates a document that stores your PDF text
docs = [Document(page_content=text)]

# Create the splitter
#each text piece can have up to 800 characters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
)

# Split into chunks
split_docs = text_splitter.split_documents(docs)

print("Total chunks:", len(split_docs))
print("First chunk:\n", split_docs[0].page_content[:500])


Total chunks: 4
First chunk:
 SolviTech — AI & Technology Consulting Proposal  
1. Company Snapshot  
SolviTech  is a technology and AI consulting firm that helps businesses automate operations, 
improve digital presence, and scale customer acquisition  using practical, result -driven AI 
solutions.  
2. Vision & Mission  
Vision  Mission  
Empower businesses with affordable, 
intelligent, scalable  technology 
solutions.  Deliver reliable AI and tech solutions  that automate 
workflows, generate leads, and support sustainab


In [13]:
!pip install -q langchain-community faiss-cpu sentence-transformers


You are creating an object called embeddings.

Inside, you tell it which model to use:
"sentence-transformers/all-MiniLM-L6-v2".

What this object does:

You give it a sentence or chunk of text →

It returns a vector (a long list of numbers) that represents the meaning of that text.

example:
Imagine each sentence is a student, and embeddings give each student a score-card of numbers describing who they are. Similar students → similar score-cards.

In [14]:
#Import embeddings + FAISS and build the index
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Create an embeddings model (lightweight, good for Colab)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Build the FAISS vector store from your chunks
#Stores all those vectors inside a FAISS database → vectorstore
vectorstore = FAISS.from_documents(split_docs, embeddings)

# Turn it into a retriever (k = how many chunks to fetch per question)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

print("Vector store built! ✅")


/tmp/ipython-input-2318656297.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store built! ✅


In [15]:
!pip install -q langchain
!pip install -qU "langchain>=0.3" "langchain-community>=0.3" langchain-core


In [16]:
!pip install -qU "langchain==0.2.14" "langchain-community==0.2.12"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 1.1.1 requires langchain-core<2.0.0,>=1.1.0, but you have langchain-core 0.2.43 which is incompatible.
langchain-classic 1.0.1 requires langchain-core<2.0.0,>=1.2.5, but you have langchain-core 0.2.43 which is incompatible.
langchain-classic 1.0.1 requ

In [17]:
# Simple in-memory chat history(object)
#It creates a simple list named chat_history to store past messages as (role, text) pairs so the program can remember the conversation.
chat_history = []  # list of (role, text) tuples, e.g. ("user", "..."), ("assistant", "...")


In [18]:
#Helper to format chat history for the prompt
def format_chat_history(history):
    if not history:
        return "No previous conversation."
    lines = []
    for role, text in history:
        lines.append(f"{role.capitalize()}: {text}")
    return "\n".join(lines)


In [19]:
#Define system instructions for the assistant

SYSTEM_INSTRUCTIONS = """
You are a helpful Product Support Assistant for a single business proposal document.
You must answer ONLY using the information from the provided context.

If the answer is NOT clearly supported by the context, say exactly:
"I couldn’t find this information in the available documents."

Be concise and clear.
"""


In [23]:
def answer_with_rag(question: str) -> str:
    global chat_history

    # 1) Retrieve relevant chunks directly from the FAISS vector store
    #    instead of using retriever.get_relevant_documents(...)
    docs = vectorstore.similarity_search(question, k=4)

    if not docs:
        answer = "I couldn’t find this information in the available documents."
        chat_history.append(("user", question))
        chat_history.append(("assistant", answer))
        return answer

    context_text = "\n\n".join(d.page_content for d in docs)

    # 2) Prepare chat history text for the prompt
    history_text = format_chat_history(chat_history)

    # 3) Build the full prompt
    prompt = f"""{SYSTEM_INSTRUCTIONS}

Context from the document:
{context_text}

Chat history:
{history_text}

User question: {question}

Answer:"""

    # 4) Call your Groq LLM
    response = llm.invoke(prompt)   # ChatGroq -> AIMessage
    try:
        answer = response.content
    except AttributeError:
        answer = str(response)

    # 5) Update simple memory
    chat_history.append(("user", question))
    chat_history.append(("assistant", answer))

    return answer


In [24]:
print(answer_with_rag("What is the main objective of this business proposal?"))


The main objective of this business proposal is to empower businesses with affordable, intelligent, scalable technology solutions, and deliver reliable AI and tech solutions that automate workflows, generate leads, and support sustainable growth.


In [28]:
print(answer_with_rag("what do you offer"))

We offer consulting and delivery services, including: 
• Website Design & Development 
• AI Chatbots for Customer Support & Sales
• Business & School Task Automation
• Lead Generation Systems & CRM Integration
• AI Solutions for Business Operations
• Ecommerce Automation & Optimization
• Custom Software Development & AI Integration


In [29]:
# 1) Global chat history
chat_history = []

# 2) The helper function
def format_chat_history(history):
    if not history:
        return "No previous conversation."
    lines = []
    for role, text in history:
        lines.append(f"{role.capitalize()}: {text}")
    return "\n".join(lines)

# 3) System instructions
SYSTEM_INSTRUCTIONS = """
You are a helpful Product Support Assistant for a single business proposal document.
You must answer ONLY using the information from the provided context.

If the answer is NOT clearly supported by the context, say exactly:
"I couldn’t find this information in the available documents."

Be concise and clear.
"""

# 4) Your Groq model
# llm = ChatGroq(...)

# 5) vectorstore built from your chunks
# vectorstore = FAISS.from_documents(split_docs, embeddings)

# 6) The latest version of answer_with_rag using vectorstore.similarity_search
def answer_with_rag(question: str) -> str:
    global chat_history

    docs = vectorstore.similarity_search(question, k=4)

    if not docs:
        answer = "I couldn’t find this information in the available documents."
        chat_history.append(("user", question))
        chat_history.append(("assistant", answer))
        return answer

    context_text = "\n\n".join(d.page_content for d in docs)
    history_text = format_chat_history(chat_history)

    prompt = f"""{SYSTEM_INSTRUCTIONS}

Context from the document:
{context_text}

Chat history:
{history_text}

User question: {question}

Answer:"""

    response = llm.invoke(prompt)
    try:
        answer = response.content
    except AttributeError:
        answer = str(response)

    chat_history.append(("user", question))
    chat_history.append(("assistant", answer))

    return answer


In [30]:
print(answer_with_rag("what do you offer"))

We offer consulting and delivery services, including: 
• Website Design & Development 
• AI Chatbots for Customer Support & Sales 
• Business & School Task Automation 
• Lead Generation Systems & CRM Integration 
• AI Solutions for Business Operations 
• Ecommerce Automation & Optimization 
• Custom Software Development & AI Integration


In [31]:
!pip install -q gradio


In [32]:
import gradio as gr

def gradio_qa(message):
    """
    Simple wrapper: takes user text and returns the RAG answer.
    Conversation memory is handled inside answer_with_rag via chat_history.
    """
    return answer_with_rag(message)

demo = gr.Interface(
    fn=gradio_qa,
    inputs=gr.Textbox(lines=2, label="Ask about the business proposal"),
    outputs=gr.Textbox(label="Assistant"),
    title="Business Proposal Support Assistant",
    description="Ask questions about the business proposal PDF. Answers come only from that document.",
)

demo.launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>